In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


**2번째 구현하는 모델은 선별한 필드만 사용해 학습한다**

In [4]:
#필요 없는 열은 제거
import pandas as pd
import numpy as np
import tensorflow as tf

data = pd.read_csv("/gdrive/My Drive/classification/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_sample = data.loc[:, ['Init_Win_bytes_forward', ' Avg Fwd Segment Size',' Init_Win_bytes_backward', ' Fwd Header Length.1',' Label']]
df_labels= df_sample[' Label']
df_packets=df_sample.drop([' Label'],axis='columns')

#같은 길이 나오는 거확인
print(len(df_sample))
print(len(df_labels))
print(len(df_packets))

#콜롬 목록과 데이터 타입 확인 후 변환
df_packets.columns
#type(df_sample)
df_packets = df_packets.astype(float) #형변환


#리스트로 되돌리기
packets= df_packets.values.tolist()
labels= df_labels.values.tolist()



#확인
packets[1]
labels[1]

#길이 확인
len(packets)
len(labels)
#len(packets[1]) #한개당 80차원

#labels도 int로 바꿔주기
for c in range(225745):
  if (labels[c] == 'BENIGN' ):
    labels[c]=0 #false
  else:
    labels[c] =1 #true
 

#테스트 군과 학습군으로 나누기.
#전체는 225745개. (70%는 약 158,021 개)
train_set = packets[:158021]
test_set = packets[158021:]
train_label =labels[:158021]
test_label = labels[158021:]

train_set = np.asarray(train_set)
train_label = np.asarray(train_label)
test_set = np.asarray(test_set)
test_label = np.asarray(test_label)


#잘 나눠졌는지 확인
print(len(train_set) + len(test_set))
print(len(test_label) + len(train_label))


#모델 만들기
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2), activation='relu', input_shape=(4, 1)),
  tf.keras.layers.MaxPooling1D(2,padding='same'),
  tf.keras.layers.Conv1D(16, (2), activation='relu'),
  tf.keras.layers.MaxPooling1D(2, padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(15, activation='softmax')
])
model.summary() #모델 출력


train_set = np.expand_dims(train_set, axis=2) #reshape
test_set = np.expand_dims(test_set, axis=2) #reshape





225745
225745
225745
225745
225745
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 3, 16)             48        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2, 16)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 16)             528       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 16)             0         
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)      

In [5]:
#학습과 평가
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_set, train_label, epochs=5, #10번의 epoch
                    validation_data=(test_set, test_label))
test_loss = model.evaluate(test_set, test_label)

Epoch 1/5
4939/4939 [==============================] - 31s 6ms/step - loss: 0.2750 - accuracy: 0.9737 - val_loss: 0.1012 - val_accuracy: 0.9763
Epoch 2/5
4939/4939 [==============================] - 31s 6ms/step - loss: 0.0429 - accuracy: 0.9927 - val_loss: 0.1146 - val_accuracy: 0.9894
Epoch 3/5
4939/4939 [==============================] - 29s 6ms/step - loss: 0.1101 - accuracy: 0.9600 - val_loss: 0.0830 - val_accuracy: 0.9822
Epoch 4/5
4939/4939 [==============================] - 28s 6ms/step - loss: 0.0305 - accuracy: 0.9940 - val_loss: 0.0457 - val_accuracy: 0.9927
Epoch 5/5
2117/2117 [==============================] - 4s 2ms/step - loss: 0.1301 - accuracy: 0.9903
